In [ ]:
import os
import json
import pandas as pd
import statistics
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
targetDir = '/home/alejandro/jRAPL/tests/dacapo/async-monitors/jolteon-results-subset'
os.chdir(targetDir)
files = sorted([ f for f in os.listdir() if f.endswith('.stats.json') ])
#files

In [ ]:
data = []
for fname in files:
    with open(fname) as f:
        parts = fname.split('.')[0].split('_')
        benchmark = parts[0]; iteration = int(parts[1]); montype = parts[2]
        
        d = json.loads(f.read())['metadata']
        d['benchmark'] = benchmark; d['iter'] = iteration; d['type'] = montype
        data.append(d)
    
#data

In [ ]:
x = {}
for d in data:
    bench = d['benchmark']
    del d['benchmark']
    if not bench in x.keys(): x[bench] = [d]
    else: x[bench].append(d)
data = x
#data

In [ ]:
#benchmarks = list(set([ f.split('_')[0] for f in files]))
#benchmarks

In [ ]:
res = {}
for benchmark in data:
    for t in ['java','c-linklist','c-dynamicarray']:
        av_samples = statistics.mean( [ d['numSamples'] for d in data[benchmark] if d['type'] == t] )
        av_lifetime = statistics.mean( [ d['lifetime'] for d in data[benchmark] if d['type'] == t] )
        
        if not benchmark in res.keys(): res[benchmark] = dict()
        if not t in res[benchmark].keys(): res[benchmark][t] = dict()
        
        res[benchmark][t]['avg_samples'] = av_samples
        res[benchmark][t]['avg_lifetime'] = av_lifetime

res

In [ ]:
labels = []
java = []
c_da = []
c_ll = []
bar_width = 0.25
for d in data:
    labels.append(d)
    d_ = data[d]
    java.append(d_['java']['avg_samples'])
    c_ll.append(d_['c-linklist']['avg_samples'])
    c_da.append(d_['c-dynamicarray']['avg_samples'])
    

In [ ]:
mpl.rcParams['figure.dpi'] = 600
r1 = np.arange(len(c_ll))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
plt.barh(r1, c_da, bar_width, color='#003f5c', edgecolor="white", label='C Dynamic Array')
plt.barh(r2, c_ll, bar_width, color='#bc5090', edgecolor="white", label='C Linked List')
plt.barh(r3, java, bar_width, color='#ffa600', edgecolor="white", label='Java')

plt.ylabel('benchmark', fontweight='bold')
plt.xlabel('Number of samples', fontweight='bold')
plt.yticks([r + bar_width for r in range(len(c_ll))], labels)
plt.legend()
fig = plt.gcf()
fig.set_size_inches(12,25)
#plt.show()

#003f5c
#bc5090
#ffa600

In [ ]:
# Generating two suplots per benchmark for [lifetime, nSamples]
'''
for bench in res:
    data = res[bench]
    #fig = plt.figure()
    fig, (axs) = plt.subplots(1,2 ,figsize=(10,5) )
    fig.suptitle('avg num samples and avg lifetime')
    
    ax = axs.flat[0]
    montypes = list(data.keys())
    num_samples_list = [ data[t]['avg_samples'] for t in data ]
    title = bench+"_average_samples"
    ax.set_title(title)
    ax.bar(montypes,num_samples_list)
    
    ax = axs.flat[1]
    montypes = list(data.keys())
    lifetime_list = [ data[t]['avg_lifetime'] for t in data ]
    ax.bar(montypes,lifetime_list)
    ax.set_title(bench+"_avg lifetime")

    plt.savefig(bench+"_numsamples")
'''